In [9]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.retsinformation.dk/eli/retsinfo/2023/9750"
response = requests.get(url)
html_content = response.text

In [4]:
html_content

'\r\n\r\n<!DOCTYPE html>\r\n<html lang="da"\r\n      prefix="\r\n    og: http://ogp.me/ns#\r\n    eli: http://data.europa.eu/eli/ontology#\r\n    iana: http://www.iana.org/">\r\n<!-- for a list of additionally recognized default prefixes in RDFa see: https://www.w3.org/2011/rdfa-context/rdfa-1.1 -->\r\n<head>\r\n    <meta charset="utf-8" />\r\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />\r\n    <base href="/"/><link rel="shortcut icon" href="/favicon.ico"/><link rel="apple-touch-icon" sizes="57x57" href="/apple-touch-icon-57x57.png"/><link rel="apple-touch-icon" sizes="60x60" href="/apple-touch-icon-60x60.png"/><link rel="apple-touch-icon" sizes="72x72" href="/apple-touch-icon-72x72.png"/><link rel="apple-touch-icon" sizes="76x76" href="/apple-touch-icon-76x76.png"/><link rel="apple-touch-icon" sizes="114x114" href="/apple-touch-icon-114x114.png"/><link rel="apple-touch-icon" sizes="120x120" href="/apple-touch-icon-120x120.png"/><link rel

### Does not work, because website is in java

In [2]:
from selenium import webdriver

url = "https://www.retsinformation.dk/eli/retsinfo/2023/9750"
driver = webdriver.Chrome()

driver.get(url)

In [3]:
html_content = driver.page_source

In [7]:
html_content[:5000]

'<html lang="da" prefix="\n    og: http://ogp.me/ns#\n    eli: http://data.europa.eu/eli/ontology#\n    iana: http://www.iana.org/"><!-- for a list of additionally recognized default prefixes in RDFa see: https://www.w3.org/2011/rdfa-context/rdfa-1.1 --><head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n    <base href="/"><style type="text/css">svg:not(:root).svg-inline--fa {\n  overflow: visible;\n}\n\n.svg-inline--fa {\n  display: inline-block;\n  font-size: inherit;\n  height: 1em;\n  overflow: visible;\n  vertical-align: -0.125em;\n}\n.svg-inline--fa.fa-lg {\n  vertical-align: -0.225em;\n}\n.svg-inline--fa.fa-w-1 {\n  width: 0.0625em;\n}\n.svg-inline--fa.fa-w-2 {\n  width: 0.125em;\n}\n.svg-inline--fa.fa-w-3 {\n  width: 0.1875em;\n}\n.svg-inline--fa.fa-w-4 {\n  width: 0.25em;\n}\n.svg-inline--fa.fa-w-5 {\n  width: 0.3125em;\n}\n.svg-inline--fa.fa-w-6 {\n  width: 0.375em;\n}\n.svg-inline--fa.fa-w-7 {\n  wid

## Extract vejledning
Simpelt script til at scrape data fra vejledning og gemme det i en dict med vejledninges navn som key

In [58]:
soup = BeautifulSoup(html_content, 'html.parser')
from bs4 import BeautifulSoup

content_div = soup.find("div", class_="document-content")

if content_div:
    title = content_div.find("p", class_="Titel2").get_text(strip=True) if content_div.find("p", class_="Titel2") else "Default Title"
    content = content_div  # This will store the entire HTML content within 'document-content'

    data = {title: content}
else:
    print("Content not found!")


In [65]:
import time

def scrape_content(urls):
    # Dictionary to store results
    driver = webdriver.Chrome()
    result_dict = {}

    for url in urls:
        driver.get(url)
        
        # Wait for the page to load completely
        time.sleep(5)

        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")
        content_div = soup.find("div", class_="document-content")

        if content_div:
            title = content_div.find("p", class_="Titel2").get_text(strip=True) if content_div.find("p", class_="Titel2") else "Default Title"
            result_dict[title] = content_div
        else:
            print("Content not found for URL:", url)

    # Close the WebDriver
    driver.quit()

    return result_dict

# List of URLs to scrape
urls = ["https://www.retsinformation.dk/eli/retsinfo/2023/9751", "https://www.retsinformation.dk/eli/retsinfo/2023/9816", "https://www.retsinformation.dk/eli/retsinfo/2023/9788"]
result = scrape_content(urls)
print(result.keys())  # To check the titles of the scraped contents

dict_keys(['Vejledning om efterlønsbevis, udskydelsesreglen og skattefri præmie m.v.', 'Vejledning om ret til fravær med barselsdagpenge for forældre til et barn, der er født eller modtaget fra den 2. august 2022', 'Vejledning om betaling af medlems- og efterlønsbidrag til en a-kasse'])


In [83]:
from bs4 import BeautifulSoup
import re

def extract_text(bs_obj):
    # Find all <p> tags and get their text content
    paragraphs = bs_obj.find_all("p")
    
    # Concatenate the text content of each <p> tag, separated by a line break
    text = "\n".join(p.get_text(strip=True) for p in paragraphs)
    
    return text

def clean_text(text):
    # Replace two or more line breaks in a row with a double line break
    cleaned_text = re.sub(r'\n{2,}', '\n\n', text)
    return cleaned_text

text_dict = {}
for title, bs_obj in result.items():
    text = extract_text(bs_obj)
    cleaned_text = clean_text(text)
    text_dict[title] = cleaned_text

In [84]:
#Save the text_dict as a csv file with title as key and content as value
import csv
with open('data/text_dict_spaces_clean.csv', 'w') as f:
    for key in text_dict.keys():
        f.write("%s,%s\n"%(key,text_dict[key]))

### Downloading all URLs for gaeldende vejledninger

In [ ]:
from selenium import webdriver

url_list = 'https://www.retsinformation.dk/documents?dt=180&h=false&page={page}&ps=100&r=30'


In [ ]:
def extract_unique_urls(url_list):
    driver = webdriver.Chrome()
    driver.get(url_list)
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    soup = soup.find("div", class_="search-result-list")
    
    urls = set()  # Using a set to store URLs to avoid duplicates
    for div in soup.find_all('div', class_='document-entry'):
        url = div.get('about')
        if url:
            urls.add(url)
    
    driver.quit()
    
    return urls

url_set = set()
url_list = 'https://www.retsinformation.dk/documents?dt=180&h=false&page={page}&ps=100&r=30'
url_prefix = 'https://www.retsinformation.dk'

for page_no in range(0, 5):
    vejledninger = extract_unique_urls(url_list.format(page=page_no))
    complete_url = [url_prefix + url for url in vejledninger]
    url_set = url_set.union(complete_url)


In [ ]:
#Save the url_set as a csv file
import csv
with open('url_gaeldende_vejledninger_30-10-2023.csv', 'w') as f:
    for url in url_set:
        f.write("%s\n"%(url))

## Download all vejledninger

In [97]:
vejledninger = scrape_content(list(url_set))

### Obs problem
Nogle vejledninger har tilsynedeladende ikke title i "Titel2" format, men derimod bare "Titel' feks tilfældet for pulverlakering vejledning

In [115]:
vejledninger_html_dict = {}
for title, bs_obj in vejledninger.items():
    html = str(bs_obj)
    vejledninger_html_dict[title] = html
    
csv_file_path = 'data/vejledninger_html_2-11-2023.csv'

with open(csv_file_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_ALL)  # QUOTE_ALL will quote all fields
    # Write the header
    writer.writerow(['Key', 'Value'])
    # Write the dictionary items
    for key, value in vejledninger_html_dict.items():
        writer.writerow([key, value])

In [117]:
import csv

vejledninger_dict = {}
for title, bs_obj in vejledninger.items():
    text = extract_text(bs_obj)
    cleaned_text = clean_text(text)
    vejledninger_dict[title] = cleaned_text
    
csv_file_path = 'data/vejledninger_tekst_2-11-2023.csv'

with open(csv_file_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_ALL)  # QUOTE_ALL will quote all fields
    # Write the header
    writer.writerow(['Key', 'Value'])
    # Write the dictionary items
    for key, value in vejledninger_dict.items():
        writer.writerow([key, value])

In [113]:
vejledninger_dict['Default Title']

'PulverlakeringAt-vejledning D.2.4August 2001\n\nPulverlakering kan foregå manuelt eller automatisk  \n\n eller begge dele i de tilfælde, hvor den automatiske sprøjtning skal  \n\n suppleres  \n\n med manuel eftersprøjtning.\nVed pulverlakering oplades pulverpartiklerne elektrostatisk  \n\n ved en elektrode, der er placeret i sprøjtepistolen. Herefter sprøjtes  \n\n pulveret  \n\n på det jordede, ophængte metalemne, hvor det danner et jævnt lag. Malingen  \n\n hærdes ved efterfølgende opvarmning i ovn til 140-  \n\n 200 °C.\nVed større produktioner bruges ofte conveyorsystemer  \n\n med ophæng, kroge m.m., som føres gennem male- og tørreanlæg. Ophængning og  \n\n nedtagning af emner foregår som regel manuelt, afhængigt af emnernes  \n\n størrelse.\nPulvermaling\nDer findes forskellige typer af pulvermaling:\n–  \n\n \t  \n\n Epoxypulvermaling,  \n\n der indeholder højmolekylær epoxyharpiks og en hærder.\n–  \n\n \t  \n\n Polyesterpulvermaling  \n\n med triglycidylisocyanurat (TGIC) som

In [111]:
len(vejledninger_html_dict.keys())

329

In [112]:
len(url_set)

432